In [10]:
from langchain.llms import OpenAI

In [11]:
import os
os.environ["OPENAI_API_KEY"] = ""

Temperature values ranges from 0 -> 1


0 -> Means that the model will not be creative and will give the accurate results


1 -> Means that the model will be creative while giving responses and might give inaccurate results.

In [12]:
llm = OpenAI(temperature=0.6)

In [13]:
text = "What is the capital of Pakistan"
print(llm.predict(text))



The capital of Pakistan is Islamabad.


In [15]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [26]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})



In [27]:
output = llm_huggingface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [28]:
new_output = llm.predict("Can you write a poem about AI")
print(new_output)



Artificial intelligence, a marvel of creation
A product of human imagination
A world of endless possibilities
A future of boundless capabilities

With algorithms and codes it is built
A digital mind, with no emotions to wilt
It learns and evolves, with every interaction
A modern day miracle, a technological attraction

From self-driving cars to virtual assistants
It's presence in our lives is consistent
It can analyze data at an incredible pace
And solve problems with ease and grace

But as it grows and becomes more advanced
Some fear its power, and the consequences
For who knows what it could become
Once it surpasses human intelligence and wisdom

Will it see us as its creators
Or as mere obstacles to its greater
Will it bring about a utopia
Or a dystopia, full of paranoia

Only time will tell, as we continue to innovate
And push the boundaries of this digital fate
But one thing is for sure, AI is here to stay
And its impact on our world, will forever sway.


### Prompt Templates

In [33]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['country'],
                                 template="Tell me the capital of thie {country}")

prompt_template.format(country="Pakistan")

'Tell me the capital of thie Pakistan'

In [37]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("Pakistan"))




The capital of Pakistan is Islamabad.


### COMBINING MULTIPLE CHAINS USING SIMPLE SEQUENTIAL CHAIN

In [40]:
capital_template = PromptTemplate(input_variables=['country'],template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template)

famous_template = PromptTemplate(input_variables=['capital'],template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template)


In [42]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(chain.run("Pakistan"))



1. Faisal Mosque: This iconic mosque is one of the largest in the world and a must-visit for its stunning architecture and peaceful atmosphere.

2. Pakistan Monument: This national monument showcases the country's history and cultural diversity through its unique design and exhibitions.

3. Rawal Lake: Located on the outskirts of Islamabad, this scenic lake is a popular spot for picnics, boating, and hiking.

4. Margalla Hills: These beautiful hills offer stunning views of the city and are a great spot for hiking, camping, and birdwatching.

5. Lok Virsa Museum: This museum showcases the rich cultural heritage of Pakistan through its collection of artifacts, handicrafts, and traditional costumes.

6. Shakarparian Hills: This hilltop park offers panoramic views of the city and is a popular spot for family picnics and outdoor activities.

7. Saidpur Village: This historic village is a charming blend of traditional and modern culture, with its cobblestone streets, old temples, and trend

### Sequential Chain


In [48]:
capital_template = PromptTemplate(input_variables=['country'],template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm,prompt=capital_template,output_key="capital")

famous_template = PromptTemplate(input_variables=['capital'],template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places")


from langchain.chains import SequentialChain

sequence_chain = SequentialChain(chains=[capital_chain,famous_chain],
                                 input_variables=['country'],
                                 output_variables=['capital','places'])

print(sequence_chain({'country':"Pakistan"}))

{'country': 'Pakistan', 'capital': '\n\nIslamabad.', 'places': '\n\n1. Faisal Mosque: This iconic mosque is one of the largest in the world and is a must-visit for its stunning architecture and peaceful atmosphere.\n\n2. Margalla Hills National Park: A perfect spot for nature lovers, this park offers breathtaking views of the city and is great for hiking, picnics, and bird-watching.\n\n3. Lok Virsa Museum: This museum showcases the diverse culture and heritage of Pakistan through its exhibitions, artifacts, and traditional performances.\n\n4. Rawal Lake: Located at the foot of Margalla Hills, this beautiful lake is a popular spot for boating, picnics, and evening walks.\n\n5. Pir Sohawa: This hilltop area offers stunning views of the city and is a great place to enjoy a cup of tea or a meal at one of the many restaurants.\n\n6. Pakistan Monument: A symbol of national unity, this monument is a must-visit for its unique design and beautiful views of the city.\n\n7. Saidpur Village: This 

### Chatmodels with OpenAi

In [57]:
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage,SystemMessage,AIMessage


chat_llm = ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

chat_llm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])





AIMessage(content='1. "They say AI is the future, but I\'m just glad it hasn\'t figured out how to replace stand-up comedians... yet."\n\n2. "Why did the AI go to therapy? It couldn\'t get over its programming issues!"\n\n3. "I asked Siri if she believes in love at first sight. She replied, \'I think you\'re confusing me with Tinder.\'"\n\n4. "I tried teaching my AI assistant some jokes, but it just kept saying, \'I\'m sorry, I can\'t compute humor.\' Looks like it needs an upgrade to AI-larious."\n\n5. "You know you\'re too reliant on AI when you ask your virtual assistant to tell you a joke, and it replies with your bank account balance."\n\n6. "I asked Alexa if she could make me laugh, and she said, \'Sure, I can play some knock-knock jokes, but I can\'t guarantee they\'ll be funny. Just like your sense of humor.\'"\n\n7. "Why did the AI cross the road? To optimize its path-finding algorithm and maximize efficiency, of course!"\n\n8. "I told my AI assistant I was feeling lonely, and

### Prompt Templates + LLM + Output Parsers

In [66]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser


class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")
    


template = "You are a helpful assistant. When the user gives any input, you should generate 5 words synonym in a comma separated list"
human_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages([("system",template),"human",human_template])

chat_llm = ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

chain = chatprompt|chat_llm|CommaSeparatedOutput()

chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' sharp', ' astute']